In [57]:
from io import BytesIO
from pathlib import Path
from zipfile import ZipFile

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
page = 'https://explore-education-statistics.service.gov.uk/find-statistics/apprenticeships'
la_codes = ['E06000014']
ssa_tier_1_codes = ["Construction, Planning and the Built Environment"]
DATA_DIR = Path('../static/autoload-data/')

## Apprenticeship data

Get the latest apprenticeship data, which is available from a web page. First we need to find the proper link.

In [4]:
with requests.get(page) as r:
    soup = BeautifulSoup(r.text)
    zip_link = soup.find('a', string='Download all data (zip)')['href']
zip_link

'https://content.explore-education-statistics.service.gov.uk/api/releases/af26c28f-9d85-4130-8f4c-a1dd4f40c2c4/files'

Now we can download the zip file ready to extract the relevant data from it

In [5]:
with requests.get(zip_link) as r:
    zip = ZipFile(BytesIO(r.content))

We want the detailed information which has starts apprenticeships. The raw data that we need is stored in a zip file in the `supporting-files` sub-folder.

In [6]:
zip_path = next((x for x in zip.namelist() if "supporting-files/app-underlying-data-starts" in x))

with zip.open(zip_path) as f:
    with ZipFile(f) as sub_zip:
        with sub_zip.open(sub_zip.namelist()[0]) as raw_csv:
            starts_df = pd.read_csv(raw_csv)

Achievements data is in a similar location.

In [7]:
zip_path = next((x for x in zip.namelist() if "supporting-files/app-underlying-data-achievements" in x))

with zip.open(zip_path) as f:
    with ZipFile(f) as sub_zip:
        with sub_zip.open(sub_zip.namelist()[0]) as raw_csv:
            achievements_df = pd.read_csv(raw_csv)

Set up some filtering and processing functions.

In [48]:
def explode_date(df, new_col='year_start'):
    df[new_col] = df.year.astype(str).str.slice(0, 4).pipe(pd.to_datetime, format="%Y") + pd.DateOffset(months=8)
    return df
def filter_construction(df):
    return df.loc[df.ssa_tier_1.isin(ssa_tier_1_codes)]
def filter_york_learners(df):
    return df.loc[df.learner_home_la_code.isin(la_codes)]

Define some pipeline functions

In [60]:
def aggregate(df, sum_field):
    return df.pipe(
        filter_construction
    ).groupby(
        ['learner_home_la_code', 'year', 'apps_level']
    )[
        sum_field
    ].sum(
    ).to_frame(
    ).melt(
        ignore_index=False
    )
def relabel(df):
    return df.reset_index(
    ).rename(
        columns={'learner_home_la_code': 'ons_code'}
    ).pipe(
        explode_date, 'year'
    ).set_index([
        'ons_code',
        'year',
        'apps_level'
    ])

Merge the two files into one and save as a partitioned Parquet file

In [61]:

APPRENTICE_DATA = DATA_DIR / 'apprenticeships/'
pd.concat(
    [
        starts_df.pipe(aggregate, 'starts'),
        achievements_df.pipe(aggregate, 'achievements'),
    ]
).pipe(
    relabel
).to_parquet(
    APPRENTICE_DATA,
    partition_cols=['ons_code']
)

There is already a summary by `la_code`, which we'll also extract.

In [10]:
csv_path = next((x for x in zip.namelist() if "data/app-starts-since-" in x))

with zip.open(csv_path) as f:
    data = pd.read_csv(f)

In [11]:
filtered = (
  data
    .loc[data.geographic_level == 'Local authority', :]
    .drop(columns=['pcon_code', 'pcon_name', 'old_la_code', 'geographic_level'])
    .rename(columns={'new_la_code': 'geo_code', 'la_name': 'geo_name'})
)
filtered['date'] = filtered.time_period.astype(str).str.slice(0,4).apply(pd.to_datetime) + pd.DateOffset(months=8, years=1, days=-1)
export = (
  filtered
    .loc[:, ['date', 'geo_code', 'starts']]
    .melt(id_vars=['geo_code', 'date'])
)

This is saved as a single file and partitioned, if required.

In [12]:
export.to_parquet(DATA_DIR / 'data.parquet')
# export.to_parquet(DATA_DIR / 'data', partition_cols=['geo_code'])